In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
import numpy as np
import shutil
import sys  
import os
import time
from sklearn.model_selection import train_test_split

In [ ]:
dir_path = 'gdrive/Shareddrives/CS5024 Ethics Project'
sys.path.append(dir_path)

In [ ]:
# Create DF for train and test set
train_df = pd.read_csv(f'{dir_path}/DrivenDataCompetition_DataFiles/TrainingData.csv')
test_df = pd.read_csv(f'{dir_path}/DrivenDataCompetition_DataFiles/TestData.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print (train_df.shape)
print (test_df.shape)

(400277, 26)
(50064, 17)


# Data Preprocessing

In [ ]:
test_df.set_index('Unnamed: 0')
train_df.set_index('Unnamed: 0')

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,...,NaN,NaN,1.00000,NaN,NaN,KINDERGARTEN,50471.810000,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860000,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,NaN,1.00000,NaN,NaN,TEACHER,62237.130000,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300000,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166000,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109283,Professional Development,ISPD,Shared Services,Non-School,Unspecified,Instructional Coach,Other Compensation/Stipend,NO_LABEL,PreK-12 Operating,WORKSHOP PARTICIPANT,...,NaN,STAFF DEV AND INSTR MEDIA,NaN,INST STAFF TRAINING SVCS,NaN,NaN,48.620000,NaN,GENERAL FUND,STAFF DEV AND INSTR MEDIA
102430,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Base Salary/Compensation,NO_LABEL,PreK-12 Operating,SALARIES OF PART TIME EMPLOYEE,...,NaN,NaN,0.00431,"TITLE II,D",NaN,PROFESSIONAL-INSTRUCTIONAL,128.824985,INSTRUCTIONAL STAFF TRAINING,NaN,INSTRUCTIONAL STAFF
413949,Parent & Community Relations,NO_LABEL,School Reported,School,NO_LABEL,Other,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,...,NaN,NaN,1.00000,NaN,NaN,PARENT/TITLE I,4902.290000,Misc,Schoolwide Schools,NaN


In [ ]:
# set target Cols
target_cols = ['Function', 'Object_Type', 'Operating_Status', 'Position_Type',  'Pre_K', 'Reporting', 'Sharing', 'Student_Type', 'Use']
train_df_dummies = pd.DataFrame()
# drop train int cols
train_df.drop(['FTE', 'Total'], axis = 1, inplace=True)
# get dummies for target cols
col = pd.get_dummies(train_df[target_cols])
# drop target cols
train_df = train_df.drop(target_cols, axis=1)
# fill NaN with space
train_df.fillna(' ', inplace=True)
# combine all text into single col
combined = pd.DataFrame([' '.join(row) for row in train_df[train_df.columns[1:]].values])
# join combined text col with dummy labels
train_df_dummies = pd.concat([combined, col], axis = 1)
# drop test int cols
test_df.drop(['FTE', 'Total'], axis = 1, inplace=True)
# fill NaN with space
test_df.fillna(' ', inplace=True)
# combine all text into single col
test_df_cleaned = pd.DataFrame([' '.join(row) for row in test_df[test_df.columns[1:]].values])
# reset indices to original
test_df_cleaned.set_index(test_df['Unnamed: 0'])
train_df_dummies.set_index(train_df['Unnamed: 0'])

,0,Function_Aides Compensation,Function_Career & Academic Counseling,Function_Communications,Function_Curriculum Development,Function_Data Processing & Information Services,Function_Development & Fundraising,Function_Enrichment,Function_Extended Time & Tutoring,Function_Facilities & Maintenance,...,Student_Type_Special Education,Student_Type_Unspecified,Use_Business Services,Use_ISPD,Use_Instruction,Use_Leadership,Use_NO_LABEL,Use_O&M,Use_Pupil Services & Enrichment,Use_Untracked Budget Set-Aside
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
134338,Teacher-Elementary KINDERGA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
206341,CONTRACTOR SERVICES BOND EXPENDITURES BUILDING...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
326408,Personal Services - Teachers TCHER 2ND GRA...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
364634,EMPLOYEE BENEFITS TEACHER SUBS GENERAL FUND Te...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
47683,TEACHER COVERAGE FOR TEACHER TEACHER SUBS GENE...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109283,WORKSHOP PARTICIPANT CURRICULUM ...,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
102430,SALARIES OF PART TIME EMPLOYEE FEDERAL GDPG ...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
413949,School Liaison PARENT/TITLE ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
cols = col.columns

In [ ]:
# rename text col
test_df_cleaned.rename(columns={0: "text"}, inplace=True)
train_df_dummies.rename(columns={0: "text"}, inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def text_processing(df):
    stop_words = set(stopwords.words('english'))
    st = SnowballStemmer('english')
    # lower and trim spaces
    df['text'] = df['text'].apply(lambda x: x.lower().strip())
    # remove other spaces
    df['text'] = df['text'].apply(lambda x: re.sub(' +', ' ', x))
    # remove punctuation
    df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
    # remove stopwords stem
    df['text'] = df['text'].apply(lambda x: ' '.join(st.stem(text) for text in x.split() if text not in stop_words))

In [ ]:
# clean df
text_processing(train_df_dummies)
text_processing(test_df_cleaned)

# Embedding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X = train_df_dummies['text'].values
y = train_df_dummies[list(train_df_dummies.columns[1:])].values
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2,train_size =0.8)

In [ ]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=1.0, min_df=1, token_pattern='(\S+)')
    # Fit the vectorizer on the train set
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    # Transform the train, test set and return the result
    X_val_tfidf = tfidf_vectorizer.transform(X_val)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    return X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_
timer = time.time()
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, test_df_cleaned.text)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

# Logistic Regression

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

def train_logreg(X_train, y_train, C, regularisation):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.

    model = OneVsRestClassifier(LogisticRegression(penalty=regularisation, C=C, max_iter=10000)).fit(X_train, y_train)
    return model

logreg_tfidf = train_logreg(X_train_tfidf, y_train, C = 2, regularisation = 'l2')
print (f'Time for NLP training" {time.time()-timer}')
timer = time.time()
y_val_predicted_labels_logreg = logreg_tfidf.predict(X_val_tfidf)
print (f'Time for NLP inference" {time.time()-timer}')


Time for NLP training" 1387.7084062099457
Time for NLP inference" 0.6320400238037109


# XG Boost

In [ ]:
# from sklearn.multiclass import OneVsRestClassifier
# from xgboost import XGBClassifier

# def train_xgb(X_train, y_train, params):
#     """
#     X_train, y_train — training data
    
#     return: trained classifier
#     """
    
#     # Create and fit XGBoost wraped into OneVsRestClassifier.

#     model = OneVsRestClassifier(XGBClassifier(**params)).fit(X_train, y_train)
#     return model
# xgb_params = {'eta': 0.3, 
#               'max_depth': 5, 
#               'subsample': 0.8, 
#               'colsample_bytree': 0.8, 
#               'tree_method' : 'gpu_hist',
#               'objective': 'binary:logistic', 
#               'eval_metric': 'auc', 
#               'seed': 42
#              }
# xgb_tfidf = train_xgb(X_train_tfidf, y_train, xgb_params)
# y_val_predicted_labels_xgb = xgb_tfidf.predict(X_val_tfidf)

# SGD

In [ ]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import SGDClassifier
# def train_sgd(X_train, y_train):
#     """
#       X_train, y_train — training data
      
#       return: trained classifier
#     """
    
#     # Create and fit LogisticRegression wraped into OneVsRestClassifier.

#     model = OneVsRestClassifier(SGDClassifier(loss = 'log', penalty = 'l2')).fit(X_train, y_train)
#     return model

# sgd_tfidf = train_sgd(X_train_tfidf, y_train)
# print (f'Time for NLP training" {time.time()-timer}')
# timer = time.time()
# y_val_predicted_labels_sgd = sgd_tfidf.predict(X_val_tfidf)
# print (f'Time for NLP inference" {time.time()-timer}')


## Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import precision_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

In [ ]:
def print_evaluation_scores(y_test, predicted):
    
    print('Accuracy: ', accuracy_score(y_test, predicted, normalize=True))
    print('F1-score macro: ', f1_score(y_test, predicted, average='macro', labels=[1]))
    print('F1-score micro: ', f1_score(y_test, predicted, average='micro', labels=[1]))
    print('F1-score weighted: ', f1_score(y_test, predicted, average='weighted'))
    print('Precision macro: ', average_precision_score(y_test, predicted, average='macro'))
    print('Precision micro: ', average_precision_score(y_test, predicted, average='micro'))
    print('Precision weighted: ', precision_score(y_test, predicted, average='weighted', labels=[1]))
    print('Log Loss: ', log_loss(y_test, predicted, normalize=True))

    
print('Metrics')
print_evaluation_scores(y_val, y_val_predicted_labels_logreg)
# print_evaluation_scores(y_val, y_val_predicted_labels_xgb)
#  print_evaluation_scores(y_val, y_val_predicted_labels_sgd)

Metrics
Accuracy:  0.898533526531428
F1-score macro:  0.857397504456328
F1-score micro:  0.857397504456328
F1-score weighted:  0.9780481032620225
Precision macro:  0.8548790839083935
Precision micro:  0.9595717445537028
Precision weighted:  0.8809523809523809
Log Loss:  27.62320822221433


# Submission

In [ ]:
test_predictions = logreg_tfidf.predict_proba(X_test_tfidf)

In [ ]:
submission_cols = pd.read_csv(f'{dir_path}/DrivenDataCompetition_DataFiles/SubmissionFormat.csv')
cols_list = list(submission_cols.columns.values)
cols_list = cols_list[1:] ## remove the first column which is the index
submission = pd.DataFrame(test_predictions, columns=cols_list)
submission.set_index(test_df['Unnamed: 0'], inplace=True)
submission.index.name = ""
submission

,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,Function__Facilities Planning,...,Student_Type__Special Education,Student_Type__Unspecified,Use__Business Services,Use__ISPD,Use__Instruction,Use__Leadership,Use__NO_LABEL,Use__O&M,Use__Pupil Services & Enrichment,Use__Untracked Budget Set-Aside
,,,,,,,,,,,,,,,,,,,,,
180042,0.005369,0.001911,0.000062,0.000037,0.000424,0.000088,0.000996,0.000203,0.001402,0.000029,...,0.002347,0.836926,0.000200,0.003739,0.064349,0.003398,0.264019,0.006676,0.005294,0.000372
28872,0.000574,0.001182,0.000243,0.005150,0.001041,0.000150,0.227637,0.003900,0.006152,0.000040,...,0.005913,0.894910,0.003721,0.009994,0.011118,0.023945,0.047036,0.023556,0.146622,0.000244
186915,0.017363,0.000992,0.000156,0.001177,0.000616,0.000108,0.000500,0.000463,0.001457,0.000034,...,0.004487,0.282162,0.000895,0.012540,0.695857,0.008499,0.016858,0.003341,0.009452,0.000168
412396,0.011996,0.000990,0.000155,0.001122,0.000625,0.000108,0.000475,0.000489,0.001456,0.000034,...,0.004447,0.250461,0.000910,0.010479,0.693750,0.007253,0.017822,0.003547,0.008566,0.000168
427740,0.001207,0.002310,0.000585,0.002508,0.002355,0.000135,0.002673,0.000067,0.003695,0.000041,...,0.004405,0.980520,0.008272,0.040371,0.001226,0.805063,0.005581,0.037933,0.004177,0.000233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169063,0.004662,0.000559,0.000271,0.001190,0.000868,0.000236,0.002325,0.000239,0.001862,0.000044,...,0.004677,0.037555,0.003252,0.003751,0.017954,0.002785,0.883382,0.002572,0.044943,0.000341
433255,0.004662,0.000559,0.000271,0.001190,0.000868,0.000236,0.002325,0.000239,0.001862,0.000044,...,0.004677,0.037555,0.003252,0.003751,0.017954,0.002785,0.883382,0.002572,0.044943,0.000341
232204,0.004662,0.000559,0.000271,0.001190,0.000868,0.000236,0.002325,0.000239,0.001862,0.000044,...,0.004677,0.037555,0.003252,0.003751,0.017954,0.002785,0.883382,0.002572,0.044943,0.000341


In [ ]:
submission.to_csv('12072022_tfidf_logreg_Cval_2.csv')